# A degenerate diffusion equation with non-constant density

Again, we want to solve the problem 
$$ \rho w - \nabla \cdot ( \rho ( u \otimes u) \nabla w) = f \text{ in } D,$$
but this time on the circle
$$D = \{ (x,y) \in \mathbb{R}^2 \ : \ \sqrt{x^2+y^2} = 1 \}.$$ 
As before, we want to approximate the exact solution 
$$w := \exp (-6((x+0.5)^2 + y^2))  - \exp (-6((x-0.5)^2+y^2)).$$
Note, that this notebook is very similar to the the one describing the method with a constant density ([DegDiffusion_constRho](./DegDiffusion_constRho.ipynb)). Hence, we will focus on describing the differences between the two notebooks.  

In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [2]:
def calculate_rhs(u,exact,rho):
    umat = CoefficientFunction(u,dims=(2,1))
    uTen = umat*umat.trans
    exactGrad = (exact.Diff(x),exact.Diff(y))
    exactDiffusion = rho*uTen*exactGrad
    exactDiv = exactDiffusion[0].Diff(x)+exactDiffusion[1].Diff(y)
    rhs = rho*exact-exactDiv
    return rhs, exactGrad

In [3]:
du = lambda u,w: InnerProduct(u,grad(w))

In [4]:
def Solve(k,exact,rho,uC,lamb,mesh):
    fes = L2(mesh, order=k, dgjumps=True)
    w,v = fes.TnT()
    gfu = GridFunction(fes)

    n = specialcf.normal(2)
    h = specialcf.mesh_size

    jump_w = w-w.Other()
    jump_v = v-v.Other()
    avg_duw = 0.5*(du(uC,w)+du(uC,w.Other()))
    avg_duv = 0.5*(du(uC,v)+du(uC,v.Other()))


    dX = dx(bonus_intorder = 2)
    dS = ds(bonus_intorder = 2)

    rhs,exactGrad = calculate_rhs(uC,exact,rho)

    a = BilinearForm(fes,symmetric=True)
    a += rho*w*v*dx
    a += rho*du(uC,w)*du(uC,v)*dX
    a += uC*n*-rho*avg_duw*jump_v*dX(skeleton=True)
    a += uC*n*-rho*avg_duv*jump_w*dX(skeleton=True)
    a += rho*lamb*1/h*(uC*n)*(uC*n)*jump_w*jump_v*dX(skeleton=True)
    a += uC*n*-rho*du(uC,w)*v*dS(skeleton=True)
    a += uC*n*-rho*du(uC,v)*w*dS(skeleton=True)
    a += rho*lamb*1/h*(uC*n)*(uC*n)*w*v*dS(skeleton=True)

    f = LinearForm(fes)
    f += rhs*v*dX
    f += uC*n*-rho*du(uC,v)*exact*dS(skeleton=True)
    f += rho*lamb*1/h*(uC*n)*(uC*n)*exact*v*dS(skeleton=True)

    a.Assemble()
    f.Assemble()

    aInv = a.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky")
    gfu.vec[:] = 0.0
    gfu.vec.data = aInv * f.vec

    return gfu, exactGrad

In [5]:
gaussp = CoefficientFunction(exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y)))
u0 = CoefficientFunction((1,1))
u1 = CoefficientFunction((-0.75*y,0.75*x))
u2 = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))

In contrast to the previous notebook, we will consider the circle geometry $D$ defined above. 

In [41]:
n = 3
geo = SplineGeometry()
geo.AddCircle(c=(0,0),r=1,bc="circle")
ngmesh = geo.GenerateMesh(maxh=1)
for i in range(n):
    ngmesh.Refine()
mesh = Mesh(ngmesh)
mesh.Curve(7)

In [42]:
Draw(gaussp,mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2102-163-g1362a8d5f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

We define a non-constant density through 
$$ \rho_{\text{rho_coef}} = (1.75-x^2-y^2)^{\text{rho_coef}}.$$
It has a peak at the origin and gets small close to the boundary. Note that the higher the coefficient $\text{rho_coef}$ is chosen, the higher the deviation between the maximum and the minimum of the density. 

In [43]:
k = 1
lamb = 10*(k+1)**2
rho_coef = 2
rho = CoefficientFunction((1.75-x**2-y**2)**rho_coef)
uC = u2

In [44]:
Draw(rho,mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2102-163-g1362a8d5f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

In [45]:
gfu, exactGrad = Solve(k,gaussp,rho,uC,lamb,mesh)

In [46]:
Draw(gfu,mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2102-163-g1362a8d5f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

In [47]:
L2error = sqrt(Integrate((gfu-gaussp)**2,mesh))
Werror = sqrt(Integrate(rho*(gfu-gaussp)**2,mesh)) + sqrt(Integrate(rho*(du(uC,gfu)-uC*exactGrad)**2,mesh))
print("L2-error: \n", L2error)
print("W-error: \n", Werror)

L2-error: 
 0.005345981943127922
W-error: 
 0.1937177039384571


In [49]:
Werrors = []
for n in range(1,6):
    geo = SplineGeometry()
    geo.AddCircle(c=(0,0),r=1,bc="circle")
    ngmesh = geo.GenerateMesh(maxh=1)
    for i in range(n):
        ngmesh.Refine()
    mesh = Mesh(ngmesh)
    mesh.Curve(7)
    gfu, exactGrad = Solve(k,gaussp,rho,uC,lamb,mesh)
    Werror = sqrt(Integrate(rho*(gfu-gaussp)**2,mesh)) + sqrt(Integrate(rho*(du(uC,gfu)-uC*exactGrad)**2,mesh))
    Werrors.append(Werror)
    print("Mesh generated with n=",n)
    if n > 1: 
        wrate = round(log(Werrors[n-2]/Werrors[n-1])/log(2),2)
        print("W-Error:",Werror)
        print("EOC:", wrate)
    else:
        print("W-Error:",Werror)

Mesh generated with n= 1
W-Error: 0.7805268058900359
Mesh generated with n= 2
W-Error: 0.38926713456081324
EOC: 1.0
Mesh generated with n= 3
W-Error: 0.1937177039384522
EOC: 1.01
Mesh generated with n= 4
W-Error: 0.0963246096876006
EOC: 1.01
Mesh generated with n= 5
W-Error: 0.047918515875208394
EOC: 1.01
